## 트라이 자료형을 사용해야 함
1. 트라이는 트리 자료 구조의 일종
2. 어떤 문자열을 검색하는 시간 복잡도는 O(m), m: 문자열의 최대 길이
3. 문자열 검색에 특화된 자료 구조
<br>
![trie](https://user-images.githubusercontent.com/38900338/92983794-89cd3480-f4e0-11ea-96d1-ca4c901f4e8b.JPG)
<br>
4. 노드의 구성: 글자를 갖는 key(J, A 등) + 마지막 글자인지 알려주는 flag(Terminal)

In [46]:
class Node(object):
    def __init__(self, key, data=None): 
        self.key = key 
        self.data = data #마지막 글자일 경우: 단어 전체, 나머지일 경우: None
        self.children = {}
        
class Trie(object):
    def __init__(self): 
        self.head = Node(None)
        
    def insert(self, string): 
        curr_node = self.head 
        for char in string: 
            if char not in curr_node.children: 
                curr_node.children[char] = Node(char) 
                curr_node = curr_node.children[char]
        curr_node.data = string
        
    def search(self, string): 
        curr_node = self.head 
        for char in string: 
            if char in curr_node.children: 
                curr_node = curr_node.children[char] 
            else: 
                return False
        if (curr_node.data != None): 
            return True
    
    def starts_with(self, prefix): # prefix로 시작하는 단어를 BFS로 찾아 리스트로 반환
        curr_node = self.head 
        result = [] 
        subtrie = None 
        for char in prefix: 
            if char in curr_node.children: 
                curr_node = curr_node.children[char] 
                subtrie = curr_node
            else: 
                return None 
        queue = list(subtrie.children.values()) 
        while queue: 
            curr = queue.pop() 
            if curr.data != None: 
                result.append(curr.data) 
            queue += list(curr.children.values()) 
        return result

In [16]:
#정확성: 25.0 효율성: 30.0 합계: 55.0 / 100.0
#런타임 에러 있음

def solution(words, queries):
    answer = []
    for q in queries:
        cnt = 0
        if q[0] != '?':
            tmp = q[:q.index('?')]
            for w in words:
                if w[:len(tmp)] == tmp and len(w) == len(q):
                    cnt += 1
        else:
            start = 0
            end = len(q) - 1
            mid = 0
            while start <= end:
                mid = (start + end) // 2
                if q[mid] == '?' and q[mid+1] != '?':
                    mid = mid+1
                    break
                elif q[mid] != '?':
                    end = mid -1
                else:
                    start = mid + 1
            tmp = q[mid:]
            for w in words:
                if w[mid:] == tmp and len(w) == len(q):
                    cnt += 1
        answer.append(cnt)
    return answer

words = ["frodo", "front", "frost", "frozen", "frame", "kakao"]
queries = ["fro??", "????o", "fr???", "fro???", "pro?"]
solution(words, queries)

[3, 2, 4, 1, 0]

In [30]:
#정확성: 25.0 효율성: 30.0 합계: 55.0 / 100.0
#런타임 에러 해결

def solution(words, queries):
    answer = []
    for q in queries:
        cnt = 0
        if q.count('?') == len(q):
            for w in words:
                if len(w) == len(q):
                    cnt += 1
        elif q[0] != '?':
            tmp = q.split('?')[0]
            for w in words:
                if w[:len(tmp)] == tmp and len(w) == len(q):
                    cnt += 1
        else: #처음시작이 '?'
            start = 0
            end = len(q) - 1
            mid = 0
            while start <= end:
                mid = (start + end) // 2
                if q[mid] == '?' and q[mid+1] != '?':
                    mid = mid+1
                    break
                elif q[mid] != '?':
                    end = mid -1
                else:
                    start = mid + 1
            tmp = q[mid:]
            for w in words:
                if w[mid:] == tmp and len(w) == len(q):
                    cnt += 1
        answer.append(cnt)
    return answer

words = ["frodo", "front", "frost", "frozen", "frame", "kakao"]
queries = ["fro??", "?????", "fr???", "fro???", "pro?"]
solution(words, queries)

[3, 5, 4, 1, 0]

In [37]:
#정확성: 25.0 효율성: 30.0 합계: 55.0 / 100.0
#시간초과 해결 안됨

def solution(words, queries):
    answer = []
    for q in queries:
        cnt = 0
        if q[0] != '?':
            tmp = q.split('?')[0]
            for w in words:
                if w[:len(tmp)] == tmp and len(w) == len(q):
                    cnt += 1
        else: #처음시작이 '?'
            q = q[::-1] #문자열 뒤집기
            tmp = q.split('?')[0]
            for w in words:
                w = w[::-1]
                if w[:len(tmp)] == tmp and len(w) == len(q):
                    cnt += 1
        answer.append(cnt)
    return answer

words = ["frodo", "front", "frost", "frozen", "frame", "kakao"]
queries = ["fro??", "?????", "fr???", "fro???", "pro?"]
solution(words, queries)

[3, 5, 4, 1, 0]

In [28]:
#정확성: 25.0 효율성: 45.0 합계: 70.0 / 100.0
#트라이 자료형 사용
#시간이 중요할 땐, 메모리를 많이 사용

from collections import defaultdict

class Node:
    def __init__(self, char, length = None, data=None): 
        self.char = char 
        self.data = None #마지막 글자일 경우: 단어 전체, 나머지일 경우: None
        self.children = {}
        self.length = defaultdict(int) # 초기값: 0
        
class Trie:
    def __init__(self): 
        self.head = Node(None)
        
    def insert(self, string): 
        curr_node = self.head 
        curr_node.length[len(string)] += 1
        for char in string: 
            if char not in curr_node.children: 
                curr_node.children[char] = Node(char) 
            curr_node.children[char].length[len(string)] += 1
            curr_node = curr_node.children[char]
        curr_node.data = string
    
    def starts_with(self, prefix, length): # prefix로 시작하는 단어를 BFS로 찾아 리스트로 반환
        curr_node = self.head 
        for char in prefix: 
            if char in curr_node.children: 
                curr_node = curr_node.children[char] 
            else: 
                return 0 #단어 없음
        return curr_node.length[length] #해당 노드를 거쳐간 문자열 중 길이가 length인 것의 개수 반환
        
    
def solution(words, queries):
    answer = []
    trie = Trie()
    rev_trie = Trie()
    
    for word in words:
        trie.insert(word)
        rev_trie.insert(word[::-1])
        
    for q in queries:
        if q == '?'*len(q):#전체 '?'
            answer.append(trie.head.length[len(q)])
        elif q[0] == '?': #시작이 '?'
            pre = q[::-1].split('?')[0]
            answer.append(rev_trie.starts_with(pre, len(q)))
        else:    
            pre = q.split('?')[0]
            answer.append(trie.starts_with(pre, len(q)))
    return answer

words = ["frodo", "front", "frost", "frozen", "frame", "kakao"]
queries = ["fro??", "?????", "fr???", "fro???", "pro?"]
solution(words, queries)

[3, 5, 4, 1, 0]

In [29]:
from collections import defaultdict


def insert(trie, word):
    curr = trie
    while word:
        s = word[0]
        if s not in curr:
            curr[s] = [0, {}]
        curr[s][0] += 1
        curr = curr[s][1]
        word = word[1:]


def find(trie, query):
    v = 0
    curr = trie
    if len(trie) == 0:
        return 0
    
    while query:
        if query[0] == "?":
            return v
        else:
            if query[0] not in curr:
                return 0
            v = curr[query[0]][0]
            curr = curr[query[0]][1]
        query = query[1:]
    return v


def solution(words, queries):
    ans = []
    trie = defaultdict(dict)
    rev_trie = defaultdict(dict)
    len_dict = defaultdict(int)

    for word in words:
        insert(trie[len(word)], word)
        insert(rev_trie[len(word)], word[::-1])
        len_dict[len(word)] += 1

    for q in queries:
        lq = len(q)
        if q[0] == "?" and q[-1] == "?":
            ans.append(len_dict[lq])
        elif q[-1] == "?":
            ans.append(find(trie[lq], q))
        elif q[0] == "?":
            ans.append(find(rev_trie[lq], q[::-1]))

    return ans

words = ["frodo", "front", "frost", "frozen", "frame", "kakao"]
queries = ["fro??", "?????", "fr???", "fro???", "pro?"]
solution(words, queries)

[3, 5, 4, 1, 0]